# Развитие игровой индустрии

- Автор: Вермишян Валерий
- Дата: 05.05.2025

# Цели и задачи проекта

**Цель** - изучение развития игровой индустрии. 

**Задачи**:
- изучить данные
- провести их предобработку
- отфильтровать данные по заданным критериям
- провести анализ данных

# Описание данных

Данные `/datasets/new_games.csv` содержат информацию о продажах игр разных жанров и платформ, а также пользовательские и экспертные оценки игр:
- `Name` — название игры.
- `Platform` — название платформы.
- `Year of Release` — год выпуска игры.
- `Genre` — жанр игры.
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` — продажи в Европе (в миллионах проданных копий).
- `JP sales` — продажи в Японии (в миллионах проданных копий).
- `Other sales` — продажи в других странах (в миллионах проданных копий).
- `Critic Score` — оценка критиков (от 0 до 100).
- `User Score` — оценка пользователей (от 0 до 10).
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

# Содержимое проекта

1. Загрузка данных и знакомство с ними
2. Проверка ошибок в данных и их предобработка
3. Фильтрация данных
4. Категоризация данных
5. Итоговый вывод

# 1. Загрузка данных и знакомство с ними

In [1]:
# Импортируем библиотеку warnings и убираем предупреждение
import warnings
warnings.filterwarnings('ignore')
# Импортируем библиотеки pandas
import pandas as pd

In [2]:
# Выгружаем данные из датасета /datasets/new_games.csv в датафрейм games
games = pd.read_csv('new_games.csv')

In [3]:
# Выводим информацию о датафрейме
games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


In [4]:
# Выводим первые 10 строк датафрейма
games.head(10)

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,23.20,2.26,4.22,0.58,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,11.28,9.14,6.5,2.88,89.0,8.5,E
7,Wii Play,Wii,2006.0,Misc,13.96,9.18,2.93,2.84,58.0,6.6,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,14.44,6.94,4.7,2.24,87.0,8.4,E
9,Duck Hunt,NES,1984.0,Shooter,26.93,0.63,0.28,0.47,NaN,NaN,NaN


Данные состоят из 11 столбцов и 16956 строк. Данные соответствуют описанию. Названия столбцов имеют пробелы, это нужно будет исправить. 

Имеются 7 столбцов типа **object**, из которых `EU sales`, `JP sales`, `User Score` нужно будет преобразовать в числовой тип, так как эти столбцы содержат числовые данные, над которыми нужно будет проводить расчеты. 

Имеются 4 столбца типа **float64**, из которых `Year of Release` нужно будет преобразовать в целочисленный тип (этого типа будет достаточно для решения наших задач)

В некоторых столбцах присутствуют пропуски, почти везде их небольшое количество. В столбцах, связанных с рейтингами (`Critic Score`, `User Score`, `Rating`) пропусков несколько тысяч

# 2. Проверка ошибок в данных и их предобработка

In [5]:
# Выводим на экран названия столбцов
games.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

Видно, что в названиях присутствуют *большие буквы* и *пробелы*, исправим это

In [6]:
# Корректируем названия столбцов
games.columns = ['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating']

Вспомним столбцы с некорректным типом данных: 
1. `eu_sales`, `jp_sales`, `user_score` имеют тип **object**, нужно преобразовать в числовой.
2. `year_of_release` имеет тип **float64**, нужно преобразовать в целочисленный

Возможные причины, почему указан некорректный тип: 
- `eu_sales`, `jp_sales` - скорее всего изначально данные были правильного типа, но при создании таблицы их случайно преобразовали в строковый вид
- `user_score` - данные вводились вручную в виде текста
- `year_of_release` - данные вводились вручную в виде числа и преобразовывались в вещественное число

Перед преобразованием типов оценим наличие пропусков в этих столбцах, убедимся, что помимо пропусков столбцы содержат корректные значения для преобразования в необходимый нам тип данных, и решим, что будем делать с пропусками


In [7]:
# Выводим уникальные значения каждого столбца, который собираемся преобразовать в другой тип. 
# Каждый столбец в отдельной ячейке Jupyter Notebook
games['eu_sales'].unique()

array(['28.96', '3.58', '12.76', '10.93', '8.89', '2.26', '9.14', '9.18',
       '6.94', '0.63', '10.95', '7.47', '6.18', '8.03', '4.89', '8.49',
       '9.09', '0.4', '3.75', '9.2', '4.46', '2.71', '3.44', '5.14',
       '5.49', '3.9', '5.35', '3.17', '5.09', '4.24', '5.04', '5.86',
       '3.68', '4.19', '5.73', '3.59', '4.51', '2.55', '4.02', '4.37',
       '6.31', '3.45', '2.81', '2.85', '3.49', '0.01', '3.35', '2.04',
       '3.07', '3.87', '3.0', '4.82', '3.64', '2.15', '3.69', '2.65',
       '2.56', '3.11', '3.14', '1.94', '1.95', '2.47', '2.28', '3.42',
       '3.63', '2.36', '1.71', '1.85', '2.79', '1.24', '6.12', '1.53',
       '3.47', '2.24', '5.01', '2.01', '1.72', '2.07', '6.42', '3.86',
       '0.45', '3.48', '1.89', '5.75', '2.17', '1.37', '2.35', '1.18',
       '2.11', '1.88', '2.83', '2.99', '2.89', '3.27', '2.22', '2.14',
       '1.45', '1.75', '1.04', '1.77', '3.02', '2.75', '2.16', '1.9',
       '2.59', '2.2', '4.3', '0.93', '2.53', '2.52', '1.79', '1.3', '2.6',
   

In [8]:
games['jp_sales'].unique()

array(['3.77', '6.81', '3.79', '3.28', '10.22', '4.22', '6.5', '2.93',
       '4.7', '0.28', '1.93', '4.13', '7.2', '3.6', '0.24', '2.53',
       '0.98', '0.41', '3.54', '4.16', '6.04', '4.18', '3.84', '0.06',
       '0.47', '5.38', '5.32', '5.65', '1.87', '0.13', '3.12', '0.36',
       '0.11', '4.35', '0.65', '0.07', '0.08', '0.49', '0.3', '2.66',
       '2.69', '0.48', '0.38', '5.33', '1.91', '3.96', '3.1', '1.1',
       '1.2', '0.14', '2.54', '2.14', '0.81', '2.12', '0.44', '3.15',
       '1.25', '0.04', '0.0', '2.47', '2.23', '1.69', '0.01', '3.0',
       '0.02', '4.39', '1.98', '0.1', '3.81', '0.05', '2.49', '1.58',
       '3.14', '2.73', '0.66', '0.22', '3.63', '1.45', '1.31', '2.43',
       '0.7', '0.35', '1.4', '0.6', '2.26', '1.42', '1.28', '1.39',
       '0.87', '0.17', '0.94', '0.19', '0.21', '1.6', '0.16', '1.03',
       '0.25', '2.06', '1.49', '1.29', '0.09', '2.87', '0.03', '0.78',
       '0.83', '2.33', '2.02', '1.36', '1.81', '1.97', '0.91', '0.99',
       '0.95', '2.0'

In [9]:
games['user_score'].unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

In [10]:
games['year_of_release'].unique()

array([2006., 1985., 2008., 2009., 1996., 1989., 1984., 2005., 1999.,
       2007., 2010., 2013., 2004., 1990., 1988., 2002., 2001., 2011.,
       1998., 2015., 2012., 2014., 1992., 1997., 1993., 1994., 1982.,
       2016., 2003., 1986., 2000.,   nan, 1995., 1991., 1981., 1987.,
       1980., 1983.])

Пока что займемся преобразованием типов, а позже проработаем пропуски. В качестве временной меры все строковые значения в числовых столбцах заменим на пропуски

In [11]:
# Преобразовываем eu_sales, jp_sales, user_score к float. 

games[['eu_sales', 'jp_sales','user_score']] = games[['eu_sales', 'jp_sales','user_score']].apply(pd.to_numeric, errors='coerce', downcast='float')

- В столбцах `eu_sales`, `jp_sales` нет пропусков, но есть значения **unknown**, что по смыслу для нас эквивалентно пропуску. Эти значения заменим на среднее значение в зависимости от названия платформы и года выхода игры.
- В столбце `user_score` есть пропуски и есть значение **tbd**. Восстановить значение рейтинга мы никак не сможем, поэтому дадим ему значение **-1** и не будем далее учитывать в анализе этого столбца
- В столбце `year_of_release` - есть небольшое кол-во пропусков, дадим ему значение **-1** и не будем далее учитывать в анализе этого столбца

In [12]:
# Теперь считаем кол-во и долю пропусков в каждом столбце

missing = pd.DataFrame({'Пропуски': games.isna().sum(), '% пропусков': round(games.isna().sum() / len(games) * 100,2)})
print(missing)

                 Пропуски  % пропусков
name                    2         0.01
platform                0         0.00
year_of_release       275         1.62
genre                   2         0.01
na_sales                0         0.00
eu_sales                6         0.04
jp_sales                4         0.02
other_sales             0         0.00
critic_score         8714        51.39
user_score           9268        54.66
rating               6871        40.52


Пропуски есть в следующих столбцах: 
- `name` - 2 пропуска, меньше 1%
- `year_of_release` - 275 пропусков, меньше 2%
- `genre` - 2 пропуска, меньше 1%
- `eu_sales` - 6 пропусков, меньше 1%
- `jp_sales` - 4 пропуска, меньше 1%
- `critic_score` - 8714 пропуска, 51%
- `user_score` - 9268 пропусков, 54%
- `rating` - 6871 пропуска, 40%

В целом пропуски характерны почти для всех столбцов, в большей степени пропуски занимают очень маленькую долю и особо не повлияют на анализ. Однако в стобцах, связанных с рейтингом, доля пропусков очень высокая - примерно половина, это может быть связано с тем, что не все игры участвовали в рейтингах и по ним просто нет необходимой информации.

В столбцах с продажами пропуски заменю на ср значение в зависимости от названия платформы и года выхода игры. В остальных перечисленных столбцах заменю на значение индикатор "-1.0" чтобы можно было работать со столбцом как с числом, а "-1.0" просто не будем учитывать

In [13]:
# Создаем функции для определения значения продаж в зависимости от названия платформы и года выхода игры
def mean_group_eu_sales(row):
    if pd.isna(row['eu_sales']):
        group = games[(games['platform'] == row['platform']) &
                   (games['year_of_release'] == row['year_of_release'])]
        return group['eu_sales'].mean()
    else:
        return row['eu_sales']
                      
def mean_group_jp_sales(row):
    if pd.isna(row['jp_sales']):
        group = games[(games['platform'] == row['platform']) &
                   (games['year_of_release'] == row['year_of_release'])]
        return group['jp_sales'].mean()
    else:
        return row['jp_sales']
                  
# Обрабатываем пропуски в eu_sales и jp_sales
games['eu_sales'] = games.apply(mean_group_eu_sales, axis=1)
games['jp_sales'] = games.apply(mean_group_jp_sales, axis=1)

# Обрабатываем пропуски в столбцах year_of_release, critic_score, user_score, rating
games['year_of_release'].fillna(-1.0, inplace=True)
games['critic_score'].fillna(-1.0, inplace=True)
games['user_score'].fillna(-1.0, inplace=True)
games['rating'].fillna('неизвестно', inplace=True)

# Повторно считаем кол-во пропусков в каждом столбце и смотрим типы данных                   
print(games.isna().sum())
games.info()

name               2
platform           0
year_of_release    0
genre              2
na_sales           0
eu_sales           0
jp_sales           0
other_sales        0
critic_score       0
user_score         0
rating             0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16956 non-null  float64
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16956 non-null  float64
 6   jp_sales         16956 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     16956 non-null  float64
 9   user_score       16956 non-null  float32
 10  rating           16956 non-null  object 
dtypes: float32(1), float64(6), object(4)
memory 

In [14]:
# Выводим уникальные значения в категориальных данных

for col in ['platform', 'year_of_release', 'genre', 'rating']:
    print(f'Уникальные значения {col}: {games[col].unique()}', end='\n\n')

Уникальные значения platform: ['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']

Уникальные значения year_of_release: [ 2.006e+03  1.985e+03  2.008e+03  2.009e+03  1.996e+03  1.989e+03
  1.984e+03  2.005e+03  1.999e+03  2.007e+03  2.010e+03  2.013e+03
  2.004e+03  1.990e+03  1.988e+03  2.002e+03  2.001e+03  2.011e+03
  1.998e+03  2.015e+03  2.012e+03  2.014e+03  1.992e+03  1.997e+03
  1.993e+03  1.994e+03  1.982e+03  2.016e+03  2.003e+03  1.986e+03
  2.000e+03 -1.000e+00  1.995e+03  1.991e+03  1.981e+03  1.987e+03
  1.980e+03  1.983e+03]

Уникальные значения genre: ['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' nan 'MISC'
 'ROLE-PLAYING' 'RACING' 'ACTION' 'SHOOTER' 'FIGHTING' 'SPORTS' 'PLATFORM'
 'ADVENTURE' 'SIMULATION' 'PUZZLE' 'STRATEGY']

Уникальные значения rating: [

- Среди `platform`, `year_of_release`, нет неявных дубликатов
- В `rating` значение **K-A** - это аналог значения **E**. Этот неявный дубликат позже устраним
- В `genre` есть много неявных дубликатов, которые тоже подлежат устранению

In [15]:
# Приводим к нижнему регистру названия жанров и проверяем уникальные значения
games['genre'] = games['genre'].str.lower()
print('Уникальные значения genre:', games['genre'].unique(), end='\n\n')

# Заменяем рейтинг K-A на E и проверяем уникальные значения
games['rating'].replace(to_replace='K-A', value='E', inplace=True)
print('Уникальные значения rating:', games['rating'].unique())

Уникальные значения genre: ['sports' 'platform' 'racing' 'role-playing' 'puzzle' 'misc' 'shooter'
 'simulation' 'action' 'fighting' 'adventure' 'strategy' nan]

Уникальные значения rating: ['E' 'неизвестно' 'M' 'T' 'E10+' 'AO' 'EC' 'RP']


Теперь значения уникальные и корректные

In [16]:
# Проверяем наличие явных дубликатов
games.duplicated().sum()

241

Найден 241 явный дубликат, удалим эти строки, сохранив датафрейм без дубликатов в новую переменную

In [17]:
# удаляем дубликаты и создаем новый датафрейм
games_no_dp = games.drop_duplicates(subset=None, keep='first', inplace=False)

In [18]:
# проверяем кол-во строк в изначальном и новом датафреймах
print('Кол-во строк до удаления дубликатов:', games.shape[0])
print('Кол-во строк после удаления дубликатов:', games_no_dp.shape[0])
count_deleted_rows = games.shape[0] - games_no_dp.shape[0]
# считаем кол-во удаленных строк в абсолютном и относительном значениях
print('Кол-во удаленных строк:', count_deleted_rows)
print('% удаленных строк: ', round((count_deleted_rows / games.shape[0] * 100),2), '%', sep='')

Кол-во строк до удаления дубликатов: 16956
Кол-во строк после удаления дубликатов: 16715
Кол-во удаленных строк: 241
% удаленных строк: 1.42%


**Промежуточный вывод:**

Проведена предобработка данных. В датафрейме имелись пропуски, столбцы с неоптимальными типами данных, неявные и явные дубликаты. На текущий момент остались пропуски в некоторых столбцах, которые нет необходимости преобразовывать во что-то конкретное. Все остальные недочеты устранены

# 3. Фильтрация данных

In [19]:
# Фильтруем игры, изданные с 2000 по 2013 год включительно
df_actual = games_no_dp.loc[(games_no_dp['year_of_release'] >= 2000) & (games_no_dp['year_of_release'] <= 2013)]

# 4. Категоризация данных

Разделим все игры по оценкам критиков и выделим такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [20]:
# Создаем функции для категоризации
def categorize_user_score(value):
    value = value['user_score']
    if value >= 8 and value <= 10:
        return '1. высокая оценка'
    elif value >= 3 and value < 8:
        return '2. средняя оценка'
    elif value >= 0 and value < 3:
        return '3. низкая оценка'
    else:
        return '4. нет оценки';
    
def categorize_critic_score(value):
    value = value['critic_score']
    if value >= 80 and value <= 100:
        return '1. высокая оценка'
    elif value >= 30 and value < 80:
        return '2. средняя оценка'
    elif value >= 0 and value < 30:
        return '3. низкая оценка'
    else:
        return '4. нет оценки';

# Категоризируем
df_actual['categorize_user_score'] = df_actual.apply(categorize_user_score, axis=1)
df_actual['categorize_critic_score'] = df_actual.apply(categorize_critic_score, axis=1)
# Проверяем результат
df_actual.head(10)

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,categorize_user_score,categorize_critic_score
0,Wii Sports,Wii,2006.0,sports,41.36,28.959999,3.77,8.45,76.0,8.0,E,1. высокая оценка,2. средняя оценка
2,Mario Kart Wii,Wii,2008.0,racing,15.68,12.760000,3.79,3.29,82.0,8.3,E,1. высокая оценка,1. высокая оценка
3,Wii Sports Resort,Wii,2009.0,sports,15.61,10.930000,3.28,2.95,80.0,8.0,E,1. высокая оценка,1. высокая оценка
6,New Super Mario Bros.,DS,2006.0,platform,11.28,9.140000,6.50,2.88,89.0,8.5,E,1. высокая оценка,1. высокая оценка
7,Wii Play,Wii,2006.0,misc,13.96,9.180000,2.93,2.84,58.0,6.6,E,2. средняя оценка,2. средняя оценка
8,New Super Mario Bros. Wii,Wii,2009.0,platform,14.44,6.940000,4.70,2.24,87.0,8.4,E,1. высокая оценка,1. высокая оценка
10,Nintendogs,DS,2005.0,simulation,9.05,10.950000,1.93,2.74,-1.0,-1.0,неизвестно,4. нет оценки,4. нет оценки
11,Mario Kart DS,DS,2005.0,racing,9.71,7.470000,4.13,1.90,91.0,8.6,E,1. высокая оценка,1. высокая оценка
13,Wii Fit,Wii,2007.0,sports,8.92,8.030000,3.60,2.15,80.0,7.7,E,2. средняя оценка,1. высокая оценка
14,Kinect Adventures!,X360,2010.0,misc,15.00,4.890000,0.24,1.69,61.0,6.3,E,2. средняя оценка,2. средняя оценка


In [21]:
# Группируем данные по выделенным категориям
grouped_data_user_score = df_actual.groupby('categorize_user_score')['name'].count()
print(grouped_data_user_score, '\n')

grouped_data_critic_score = df_actual.groupby('categorize_critic_score')['name'].count()
print(grouped_data_critic_score)

categorize_user_score
1. высокая оценка    2286
2. средняя оценка    4081
3. низкая оценка      116
4. нет оценки        6298
Name: name, dtype: int64 

categorize_critic_score
1. высокая оценка    1692
2. средняя оценка    5422
3. низкая оценка       55
4. нет оценки        5612
Name: name, dtype: int64


In [22]:
# Топ-7 платформ по кол-ву игр
grouped_data_top = df_actual.groupby('platform')['name'].count().sort_values(ascending=False)
print(grouped_data_top.head(7))

platform
PS2     2127
DS      2120
Wii     1275
PSP     1180
X360    1121
PS3     1087
GBA      811
Name: name, dtype: int64


# 5. Итоговый вывод

В рамках проекта была проведена работа с датасетом `/datasets/new_games.csv`

1. Была изучена информация, которая в нем хранится: какие в нем столбцы, типы данных, какие значения содержатся, есть ли пропуски и явные/неявные дубликаты
2. Была проведена предобработка датасета: устранены дубликаты, некоторые столбцы преобразованы в корректный тип данных, пропуски приведены к единому виду
3. Из датасета выделен необходимый для анализа срез данных: игры, созданные с 2000 по 2013 год включительно
4. Добавлена категоризация по рейтингу пользователей и рейтингу критиков: в датасете появилось два новых поля: `categorize_user_score` и `categorize_critic_score`
5. Проведена группировка по полученным категориям